In [11]:
from astropy.io import ascii
from astropy.io import fits
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage

In [12]:
datadirpath = "./data/005_center_multishift/images/"
datafilenum = 52
datafilename = "wl_channel_%05i.fits"%(datafilenum)
datafilepath = datadirpath + datafilename

outdirname = "out/"
outmedpsffilename = "data_%05i_medpsf.fits"%(datafilenum)
outrotfilename = "data_%05i_rot.fits"%(datafilenum)
outsumfilename = "data_%05i_sum.fits"%(datafilenum)
outmeanfilename = "data_%05i_mean.fits"%(datafilenum)

outmedpsffilepath = outdirname + outmedpsffilename
outrotfilepath = outdirname + outrotfilename
outsumfilepath = outdirname + outsumfilename
outmeanfilepath = outdirname + outmeanfilename

anglesfilepath = "data/parangs_bads_removed.txt"


In [13]:
def savedata(data, path, overwrite=True):
    hdul = fits.HDUList()
    hdul.append(fits.PrimaryHDU())
    hdul.append(fits.ImageHDU(data=data))
    hdul.writeto(path, overwrite=overwrite)

In [14]:
anglestable = ascii.read(anglesfilepath, format="no_header", data_start=0)
angles = anglestable['col1'].data

In [15]:
with fits.open(datafilepath) as datahdu:
    data = datahdu[0].data

In [16]:
medpsf = np.median(data, axis=0)
savedata(medpsf, outmedpsffilepath)

In [17]:
datasub = np.subtract(data, medpsf)

In [18]:
assert len(angles) == len(datasub)
datarot = np.empty(shape=datasub.shape)
for i in range(len(angles)):
    datarot[i] = ndimage.rotate(datasub[i], -1 * angles[i], reshape=False)

savedata(datarot, outrotfilepath)

In [ ]:
datasum = np.sum(datarot, axis=0)
savedata(datasum, outsumfilepath)

In [ ]:
datamean = np.mean(datarot, axis=0)
savedata(datamean, outmeanfilepath)